# Copernicus Sentinel-2 Data Search and Exploration

This Jupyter Notebook demonstrates how to search, filter, and explore Sentinel-2 satellite imagery metadata using the `phisenapi` library. The workflow includes:

- **Setting Search Parameters:**  
    The notebook defines parameters such as collection name (`SENTINEL-2`), product type (`S2MSI1C`), cloud cover threshold, area of interest (AOI) in WKT format, date range, and sorting options.

- **Executing the Search:**  
    An instance of `CopernicusDataSearcher` is configured with the specified parameters and used to query the Copernicus Open Access Hub for matching Sentinel-2 products.

- **Viewing Results:**  
    The resulting metadata is loaded into a pandas DataFrame (`df`). Key columns such as `Id`, `Name`, `S3Path`, and `GeoFootprint` are displayed for quick inspection.

## Variables and Data

- **aoi_wkt:** AOI polygon in WKT format. Polygon must start and end with the same point. Coordinates must be given in EPSG 4326
- **cloud_cover_threshold:** Maximum allowed cloud cover percentage.
- **collection_name:** Data collection to search (`SENTINEL-2`).
- **columns_to_display:** List of DataFrame columns to display.
- **df:** DataFrame containing search results (1000 entries, 16 columns).
- **searcher:** Configured `CopernicusDataSearcher` instance.
- **start_date, end_date:** Date range for the search.
- **product_type, orbit_direction, order_by, top:** Additional search parameters.

In [ ]:
%load_ext autoreload
%autoreload 2

# Search

You can modify the search parameters to refine your query, re-run the search, and further analyze or visualize the results as needed.

In [1]:
from phidown.search import CopernicusDataSearcher
# You can modify these parameters as needed

searcher = CopernicusDataSearcher(
    collection_names=['SENTINEL-1'],
    product_type='SLC',
    orbit_direction=None,
    cloud_cover_threshold=None,
    start_date='2022-05-03T00:00:00.000Z',
    end_date='2022-05-03T04:00:00.000Z',
    top=3,
)
df = searcher.execute_query()
searcher.display_results()

,Id,Name,S3Path,GeoFootprint,OriginDate,Attributes
0,0720d51b-aad3-5972-a41f-9fb5a377d88c,S1A_IW_SLC__1SDV_20220503T035608_20220503T0356...,/eodata/Sentinel-1/SAR/SLC/2022/05/03/S1A_IW_S...,"{'type': 'Polygon', 'coordinates': [[[25.95559...",2022-05-03 09:09:06,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
1,ea2651cc-cbc5-56da-965e-0742b927cfc6,S1A_IW_SLC__1SDV_20220503T035542_20220503T0356...,/eodata/Sentinel-1/SAR/SLC/2022/05/03/S1A_IW_S...,"{'type': 'Polygon', 'coordinates': [[[26.37545...",2022-05-03 09:24:00,"[{'@odata.type': '#OData.CSC.StringAttribute',..."
2,6bcec001-b3ac-5537-bb33-b52ec097f46d,S1A_IW_SLC__1SDV_20220503T035517_20220503T0355...,/eodata/Sentinel-1/SAR/SLC/2022/05/03/S1A_IW_S...,"{'type': 'Polygon', 'coordinates': [[[26.72352...",2022-05-03 09:15:13,"[{'@odata.type': '#OData.CSC.StringAttribute',..."


In [2]:
for product_name in df['Name']:
    print(f"Downloading {product_name}...")
    # searcher.download_product(product_name, output_dir='.', overwrite=True)

In [ ]:
filename = 'S1A_IW_SLC__1SDV_20220503T034102_20220503T034129_043041_0523B0_16A0.SAFE'
searcher.download_product(
    filename,
)

Access token saved to disk.
Temporary S3 credentials created successfully.
access: 1KTF7U79MH0GNQPBJ2EZ
secret: uOriDQI0UmLet2O89x3RV6B1nRLDq1iCnDZSaGhC


.                                       |                    |   0% 0.00/0.00B


Failed to download Sentinel-1/SAR/SLC/2022/05/03/S1A_IW_SLC__1SDV_20220503T034102_20220503T034129_043041_0523B0_16A0.SAFE/. Error: [Errno 22] Invalid argument: 'S1A_IW_SLC__1SDV_20220503T034102_20220503T034129_043041_0523B0_16A0.SAFE/..8dc1c0e6' -> 'S1A_IW_SLC__1SDV_20220503T034102_20220503T034129_043041_0523B0_16A0.SAFE/.'


S1A_IW_SLC__1SDV_20220503T034102_2022...|████████████████████| 100% 16.1k/16.1kB
annotation                              |                    |   0% 0.00/0.00B


Creating a directory: S1A_IW_SLC__1SDV_20220503T034102_20220503T034129_043041_0523B0_16A0.SAFE/annotation


...

calibration-s1a-iw1-slc-vh-20220503t0...|████████████████████| 100% 935k/935kB
calibration-s1a-iw1-slc-vv-20220503t0...|████████████████████| 100% 935k/935kB
calibration-s1a-iw2-slc-vh-20220503t0...|████████████████████| 100% 1.10M/1.10MB
calibration-s1a-iw2-slc-vv-20220503t0...|████████████████████| 100% 1.10M/1.10MB
calibration-s1a-iw3-slc-vh-20220503t0...|████████████████████| 100% 1.06M/1.06MB
calibration-s1a-iw3-slc-vv-20220503t0...|████████████████████| 100% 1.06M/1.06MB
noise-s1a-iw1-slc-vh-20220503t034103-...|████████████████████| 100% 131k/131kB
noise-s1a-iw1-slc-vv-20220503t034103-...|████████████████████| 100% 131k/131kB
noise-s1a-iw2-slc-vh-20220503t034104-...|████████████████████| 100% 145k/145kB
noise-s1a-iw2-slc-vv-20220503t034104-...|████████████████████| 100% 145k/145kB
noise-s1a-iw3-slc-vh-20220503t034102-...|████████████████████| 100% 145k/145kB
noise-s1a-iw3-slc-vv-20220503t034102-...|████████████████████| 100% 145k/145kB
rfi                                     |   

Creating a directory: S1A_IW_SLC__1SDV_20220503T034102_20220503T034129_043041_0523B0_16A0.SAFE/annotation/rfi


rfi-s1a-iw1-slc-vh-20220503t034103-20...|████████████████████| 100% 14.8k/14.8kB
rfi-s1a-iw1-slc-vv-20220503t034103-20...|████████████████████| 100% 6.42k/6.42kB
rfi-s1a-iw2-slc-vh-20220503t034104-20...|████████████████████| 100% 15.1k/15.1kB
rfi-s1a-iw2-slc-vv-20220503t034104-20...|████████████████████| 100% 6.76k/6.76kB
rfi-s1a-iw3-slc-vh-20220503t034102-20...|████████████████████| 100% 15.1k/15.1kB
rfi-s1a-iw3-slc-vv-20220503t034102-20...|████████████████████| 100% 6.76k/6.76kB
s1a-iw1-slc-vh-20220503t034103-202205...|████████████████████| 100% 916k/916kB
s1a-iw1-slc-vv-20220503t034103-202205...|████████████████████| 100% 916k/916kB
s1a-iw2-slc-vh-20220503t034104-202205...|████████████████████| 100% 906k/906kB
s1a-iw2-slc-vv-20220503t034104-202205...|████████████████████| 100% 906k/906kB
s1a-iw3-slc-vh-20220503t034102-202205...|████████████████████| 100% 817k/817kB
s1a-iw3-slc-vv-20220503t034102-202205...|████████████████████| 100% 817k/817kB
manifest.safe                           

Creating a directory: S1A_IW_SLC__1SDV_20220503T034102_20220503T034129_043041_0523B0_16A0.SAFE/measurement


s1a-iw1-slc-vh-20220503t034103-202205...|████████████████████| 100% 1.20G/1.20GB
s1a-iw1-slc-vv-20220503t034103-202205...|██████▍             |  32% 382M/1.20GB 